## Importing Libraries
---

In [1]:
import numpy as np
import pandas as pd
import json
pd.set_option("display.max_colwidth", None)

In [2]:
with open('/kaggle/input/chatbots-intent-recognition-dataset/Intent.json') as f:
    data = json.load(f)

In [3]:
df = pd.DataFrame(data['intents'])
df.head()

,intent,text,responses,extension,context,entityType,entities
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, Hya there]","[Hi human, please tell me your GeniSys user, Hello human, please tell me your GeniSys user, Hola human, please tell me your GeniSys user]","{'function': '', 'entities': False, 'responses': []}","{'in': '', 'out': 'GreetingUserRequest', 'clear': False}",NA,[]
1,GreetingResponse,"[My user is Adam, This is Adam, I am Adam, It is Adam, My user is Bella, This is Bella, I am Bella, It is Bella]","[Great! Hi <HUMAN>! How can I help?, Good! Hi <HUMAN>, how can I help you?, Cool! Hello <HUMAN>, what can I do for you?, OK! Hola <HUMAN>, how can I help you?, OK! hi <HUMAN>, what can I do for you?]","{'function': 'extensions.gHumans.updateHuman', 'entities': True, 'responses': ['Hi %%HUMAN%%! How can I help?', 'Hi %%HUMAN%%, how can I help you?', 'Hello %%HUMAN%%, what can I do for you?', 'Hola %%HUMAN%%, how can I help you?', 'OK hi %%HUMAN%%, what can I do for you?']}","{'in': 'GreetingUserRequest', 'out': '', 'clear': True}",NA,"[{'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo': 4}, {'entity': 'HUMAN', 'rangeFrom': 2, 'rangeTo': 3}, {'entity': 'HUMAN', 'rangeFrom': 1, 'rangeTo': 2}, {'entity': 'HUMAN', 'rangeFrom': 2, 'rangeTo': 3}, {'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo': 4}, {'entity': 'HUMAN', 'rangeFrom': 2, 'rangeTo': 3}, {'entity': 'HUMAN', 'rangeFrom': 1, 'rangeTo': 2}, {'entity': 'HUMAN', 'rangeFrom': 2, 'rangeTo': 3}]"
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are you?, Hola how are you?, How are you doing?, Hope you are doing well?, Hello hope you are doing well?]","[Hello, I am great, how are you? Please tell me your GeniSys user, Hello, how are you? I am great thanks! Please tell me your GeniSys user, Hello, I am good thank you, how are you? Please tell me your GeniSys user, Hi, I am great, how are you? Please tell me your GeniSys user, Hi, how are you? I am great thanks! Please tell me your GeniSys user, Hi, I am good thank you, how are you? Please tell me your GeniSys user, Hi, good thank you, how are you? Please tell me your GeniSys user]","{'function': '', 'entities': False, 'responses': []}","{'in': '', 'out': 'CourtesyGreetingUserRequest', 'clear': True}",NA,[]
3,CourtesyGreetingResponse,"[Good thanks! My user is Adam, Good thanks! This is Adam, Good thanks! I am Adam, Good thanks! It is Adam, Great thanks! My user is Bella, Great thanks! This is Bella, Great thanks! I am Bella, Great thanks! It is Bella]","[Great! Hi <HUMAN>! How can I help?, Good! Hi <HUMAN>, how can I help you?, Cool! Hello <HUMAN>, what can I do for you?, OK! Hola <HUMAN>, how can I help you?, OK! hi <HUMAN>, what can I do for you?]","{'function': 'extensions.gHumans.updateHuman', 'entities': True, 'responses': ['Great %%HUMAN%%! How can I help?', 'Good %%HUMAN%%, how can I help you?', 'Cool %%HUMAN%%, what can I do for you?', 'OK %%HUMAN%%, how can I help you?', 'OK hi %%HUMAN%%, what can I do for you?']}","{'in': 'GreetingUserRequest', 'out': '', 'clear': True}",NA,"[{'entity': 'HUMAN', 'rangeFrom': 5, 'rangeTo': 6}, {'entity': 'HUMAN', 'rangeFrom': 4, 'rangeTo': 5}, {'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo': 4}, {'entity': 'HUMAN', 'rangeFrom': 4, 'rangeTo': 5}, {'entity': 'HUMAN', 'rangeFrom': 5, 'rangeTo': 6}, {'entity': 'HUMAN', 'rangeFrom': 4, 'rangeTo': 5}, {'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo': 4}, {'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo': 4}]"
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who do you think I am?, What do you think I am?, Who are you talking to?, What name do you call me by?, Tell me my name]","[You are <HUMAN>! How can I help?, Your name is <HUMAN>, how can I help you?, They call you <HUMAN>, what can I do for you?, Your name is <HUMAN>, how can I help you?, <HUMAN>, what can I do for you?]","{'function': 'extensions.gHumans.getCurrentHuman', 'entities': False, 'responses': ['You are %%HUMAN%%! How can I help?', 'Your name is %%HUMAN%%, 

## Data Preprocessing
---

In [4]:
df_patterns = df[['text', 'intent']]
df_responses = df[['responses', 'intent']]
df_patterns.head()

,text,intent
0,"[Hi, Hi there, Hola, Hello, Hello there, Hya, Hya there]",Greeting
1,"[My user is Adam, This is Adam, I am Adam, It is Adam, My user is Bella, This is Bella, I am Bella, It is Bella]",GreetingResponse
2,"[How are you?, Hi how are you?, Hello how are you?, Hola how are you?, How are you doing?, Hope you are doing well?, Hello hope you are doing well?]",CourtesyGreeting
3,"[Good thanks! My user is Adam, Good thanks! This is Adam, Good thanks! I am Adam, Good thanks! It is Adam, Great thanks! My user is Bella, Great thanks! This is Bella, Great thanks! I am Bella, Great thanks! It is Bella]",CourtesyGreetingResponse
4,"[What is my name?, What do you call me?, Who do you think I am?, What do you think I am?, Who are you talking to?, What name do you call me by?, Tell me my name]",CurrentHumanQuery


In [5]:
df_patterns.shape

(22, 2)

In [6]:
df_patterns = df_patterns.explode('text')
df_patterns.head()

,text,intent
0,Hi,Greeting
0,Hi there,Greeting
0,Hola,Greeting
0,Hello,Greeting
0,Hello there,Greeting


In [7]:
df_patterns.describe()

,text,intent
count,143,143
unique,142,22
top,Your real name please?,CourtesyGreetingResponse
freq,2,8


In [8]:
df_patterns.drop_duplicates(inplace= True)

In [9]:
df_patterns.describe()

,text,intent
count,142,142
unique,142,22
top,Hi,CourtesyGreetingResponse
freq,1,8


In [10]:
df_patterns['intent'].value_counts()

CourtesyGreetingResponse    8
GreetingResponse            8
Greeting                    7
NotTalking2U                7
PodBayDoorResponse          7
PodBayDoor                  7
Clever                      7
Shutup                      7
TimeQuery                   7
CurrentHumanQuery           7
CourtesyGreeting            7
SelfAware                   7
Thanks                      6
UnderstandQuery             6
RealNameQuery               6
CourtesyGoodBye             6
WhoAmI                      6
NameQuery                   6
Gossip                      6
Jokes                       6
Swearing                    4
GoodBye                     4
Name: intent, dtype: int64

## Data Balancing
---
We will balance the unbalanced classes by randomly duplicating rows from such classes

In [11]:
def balance_data(df_patterns):
    df_intent = df_patterns['intent']
    max_counts = df_intent.value_counts().max() #max number of examples for a class
    
    new_df = df_patterns.copy()
    for i in df_intent.unique():
        i_count = int(df_intent[df_intent == i].value_counts())
        if i_count < max_counts:
            i_samples = df_patterns[df_intent == i].sample(max_counts - i_count, replace = True, ignore_index = True)
            new_df = pd.concat([new_df, i_samples])
    return new_df

In [12]:
df_patterns = balance_data(df_patterns)

In [13]:
df_patterns['intent'].value_counts()

Greeting                    8
GreetingResponse            8
PodBayDoorResponse          8
PodBayDoor                  8
Jokes                       8
Gossip                      8
Clever                      8
WhoAmI                      8
CourtesyGoodBye             8
GoodBye                     8
Swearing                    8
Shutup                      8
UnderstandQuery             8
NotTalking2U                8
Thanks                      8
TimeQuery                   8
RealNameQuery               8
NameQuery                   8
CurrentHumanQuery           8
CourtesyGreetingResponse    8
CourtesyGreeting            8
SelfAware                   8
Name: intent, dtype: int64

## Token_ID, Attention_Mask, Labels
---

In [14]:
seq_len = 256
num_samples = len(df_patterns)

Xids = np.zeros((num_samples, seq_len)) #Token ids
Xmask = np.zeros((num_samples, seq_len)) #attention mask

In [15]:
Xids.shape

(176, 256)

In [16]:
Xids

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
%%time
# we will populate the above zero arrays with actual values for each token
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(df_patterns['text']):
    tokens = tokenizer.encode_plus(phrase, max_length= seq_len, truncation= True, padding= 'max_length', add_special_tokens = True, return_tensors= 'tf')
    
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

CPU times: user 4.35 s, sys: 1.6 s, total: 5.96 s
Wall time: 14.7 s


In [18]:
Xids

array([[  101.,  8790.,   102., ...,     0.,     0.,     0.],
       [  101.,  8790.,  1175., ...,     0.,     0.,     0.],
       [  101.,  9800.,  1742., ...,     0.,     0.,     0.],
       ...,
       [  101., 18959.,  1181., ...,     0.,     0.,     0.],
       [  101., 13410.,  1128., ...,     0.,     0.,     0.],
       [  101.,  2825.,  1128., ...,     0.,     0.,     0.]])

one-hot encoding the labels  
first we will label encode the current values

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [20]:
arr = le.fit_transform(df_patterns['intent'])
arr

array([ 7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  2,  2,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4, 10, 10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 14, 19, 19,
       19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 11, 11, 11, 11, 11, 11,
       11, 20, 20, 20, 20, 20, 20, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17,
       17,  5,  5,  5,  5,  1,  1,  1,  1,  1,  1, 21, 21, 21, 21, 21, 21,
        0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  6,  6,  6,  9,  9,  9,  9,
        9,  9, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 15,
       15, 15, 15, 15, 15, 15,  7,  2,  4, 10, 10, 14, 14, 19, 18, 18, 11,
       20, 20, 16, 17, 17, 17, 17,  5,  5,  5,  5,  1,  1, 21, 21,  0,  6,
        6,  9,  9, 12, 13, 15])

In [21]:
le.classes_

array(['Clever', 'CourtesyGoodBye', 'CourtesyGreeting',
       'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye',
       'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery',
       'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse',
       'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks',
       'TimeQuery', 'UnderstandQuery', 'WhoAmI'], dtype=object)

In [22]:
labels = np.zeros((num_samples, arr.max()+1))
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
labels[np.arange(num_samples), arr] = 1
labels 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
len(labels)

176

## Data Pipeline
---

In [25]:
import tensorflow as tf

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(22,), dtype=tf.float64, name=None))>

In [27]:
dataset.as_numpy_iterator().next()

(array([ 101., 8790.,  102.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
      

Tensorflow expects a tuple with 2 inputs only, examples at index 0 and Labels at index 1

In [28]:
def map_function(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [29]:
dataset = dataset.map(map_function)

In [30]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(22,), dtype=tf.float64, name=None))>

In [31]:
batch_size = 8

dataset = dataset.shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder= True)

In [32]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(8, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(8, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(8, 22), dtype=tf.float64, name=None))>

In [33]:
split = 0.9 

size = int((num_samples / batch_size) * split)

In [34]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

## Training the Tranformer model
---

In [35]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [36]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [37]:
input_ids = tf.keras.layers.Input(shape= (seq_len,), name= 'input_ids', dtype='int32') #input layer for tokens
mask = tf.keras.layers.Input(shape= (seq_len,), name= 'attention_mask', dtype='int32') #input layer for attention masks

embeddings = bert.bert(input_ids, attention_mask= mask)[1]

x = tf.keras.layers.Dense(1024, activation= 'relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation= 'softmax', name= 'outputs')(x)

In [38]:
model = tf.keras.Model(inputs= [input_ids, mask], outputs = y)

In [39]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [40]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [41]:
model.compile(optimizer= optimizer, loss= loss, metrics = [acc])

In [42]:
%%time
history = model.fit(train_ds, validation_data= val_ds, epochs= 10)

Epoch 1/10
19/19 [==============================] - 48s 546ms/step - loss: 3.1598 - accuracy: 0.0789 - val_loss: 2.9018 - val_accuracy: 0.2083
Epoch 2/10
19/19 [==============================] - 8s 415ms/step - loss: 2.9587 - accuracy: 0.1776 - val_loss: 2.7512 - val_accuracy: 0.3333
Epoch 3/10
19/19 [==============================] - 8s 415ms/step - loss: 2.7536 - accuracy: 0.3421 - val_loss: 2.5247 - val_accuracy: 0.5000
Epoch 4/10
19/19 [==============================] - 8s 418ms/step - loss: 2.4459 - accuracy: 0.5066 - val_loss: 2.1476 - val_accuracy: 0.5833
Epoch 5/10
19/19 [==============================] - 8s 422ms/step - loss: 2.1873 - accuracy: 0.6579 - val_loss: 1.7853 - val_accuracy: 0.7917
Epoch 6/10
19/19 [==============================] - 8s 427ms/step - loss: 1.8657 - accuracy: 0.7763 - val_loss: 1.5190 - val_accuracy: 0.8333
Epoch 7/10
19/19 [==============================] - 8s 427ms/step - loss: 1.6325 - accuracy: 0.8421 - val_loss: 1.1650 - val_accuracy: 0.9583
Epoch

## Saving the model
---

In [43]:
model.save('intent_prediction_model.h5')

## Loding the model
---

In [44]:
loaded_model = tf.keras.models.load_model('intent_prediction_model.h5')

In [45]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [46]:
seq_len

256

## Prediction
---

In [47]:
def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length = seq_len,
                                  truncation= True, padding= 'max_length',
                                  add_special_tokens = True,
                                  return_tensors= 'tf')
    return {
        'input_ids' : tf.cast(tokens['input_ids'], tf.float64),
        'attention_mask' : tf.cast(tokens['attention_mask'], tf.float64)
    }

In [48]:
def predict_res(text):
    prep = prep_data(text)
    prob = loaded_model.predict(prep)
    res_dict = {0: 'Clever',
                 1: 'CourtesyGoodBye',
                 2: 'CourtesyGreeting',
                 3: 'CourtesyGreetingResponse',
                 4: 'CurrentHumanQuery',
                 5: 'GoodBye',
                 6: 'Gossip',
                 7: 'Greeting',
                 8: 'GreetingResponse',
                 9: 'Jokes',
                 10: 'NameQuery',
                 11: 'NotTalking2U',
                 12: 'PodBayDoor',
                 13: 'PodBayDoorResponse',
                 14: 'RealNameQuery',
                 15: 'SelfAware',
                 16: 'Shutup',
                 17: 'Swearing',
                 18: 'Thanks',
                 19: 'TimeQuery',
                 20: 'UnderstandQuery',
                 21: 'WhoAmI'}
    return res_dict[np.argmax(prob)]

In [49]:
li = ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting',
       'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye',
       'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery',
       'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse',
       'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks',
       'TimeQuery', 'UnderstandQuery', 'WhoAmI']

di = {}

for i,j in enumerate(li):
    di.update({i:j})

In [50]:
di

{0: 'Clever',
 1: 'CourtesyGoodBye',
 2: 'CourtesyGreeting',
 3: 'CourtesyGreetingResponse',
 4: 'CurrentHumanQuery',
 5: 'GoodBye',
 6: 'Gossip',
 7: 'Greeting',
 8: 'GreetingResponse',
 9: 'Jokes',
 10: 'NameQuery',
 11: 'NotTalking2U',
 12: 'PodBayDoor',
 13: 'PodBayDoorResponse',
 14: 'RealNameQuery',
 15: 'SelfAware',
 16: 'Shutup',
 17: 'Swearing',
 18: 'Thanks',
 19: 'TimeQuery',
 20: 'UnderstandQuery',
 21: 'WhoAmI'}

In [51]:
predict_res("what are you doing?")

1/1 [==============================] - 3s 3s/step


'NameQuery'

In [52]:
predict_res("Hi")

1/1 [==============================] - 0s 73ms/step


'Greeting'

In [53]:
predict_res("What is your product")

1/1 [==============================] - 0s 73ms/step


'NameQuery'

In [54]:
predict_res("You know what")

1/1 [==============================] - 0s 79ms/step


'UnderstandQuery'

In [55]:
# NameQuery
predict_res("What could I call you?")

1/1 [==============================] - 0s 78ms/step


'NameQuery'

In [56]:
# Thanks
predict_res("OK thank you")

1/1 [==============================] - 0s 100ms/step


'Thanks'

In [57]:
# TimeQuery
predict_res("Tell me what time it is?")

1/1 [==============================] - 0s 66ms/step


'TimeQuery'